In [11]:
import pandas as pd
import numpy as np

def getRT(row):
    return np.mean([row["rt_start"],row["rt_end"]])

In [12]:
peakListPosFn = "../data/IDX/AX_stab_pool/pos/inclusionList_exported.csv"
peakListNegFn = "../data/IDX/AX_stab_pool/neg/inclusionList_neg.csv"

In [13]:
peakListPos = pd.read_csv(peakListPosFn)
peakListNeg = pd.read_csv(peakListNegFn)

In [14]:
peakListPos

,Name,mz,rt_start,rt_end,Charge
0,Compound 1,524.265000,0.000000,15.000000,1
1,Compound 2,108.080841,0.828070,1.216255,1
2,Compound 3,102.127731,6.769517,7.328251,1
3,Compound 4,102.127731,6.980431,7.453215,1
4,Compound 5,162.112534,5.335723,5.680650,1
...,...,...,...,...,...
575,Compound 576,595.514160,0.606029,0.732323,1
576,Compound 577,677.297913,0.575798,0.750840,1
577,Compound 578,673.326538,0.573538,0.771442,1
578,Compound 579,308.114716,5.864152,6.283127,1


In [15]:
peakListNeg

,mz,rt_start,rt_end,Charge
0,524.265000,0.000000,15.000000,-1
1,283.264099,0.625200,0.854045,-1
2,255.233078,0.580297,0.864533,-1
3,281.248688,0.601802,0.819549,-1
4,279.233124,0.608019,0.818380,-1
...,...,...,...,...
402,269.121613,1.012359,1.097021,-1
403,324.072235,5.060811,5.288692,-1
404,190.035751,7.331674,7.617239,-1
405,238.891876,10.705984,11.104264,-1


In [22]:
rtTol = .33
ppmTol = 5

consensousPeaks = pd.DataFrame(peakListPos)
toAdd = {}
compoundInd = len(consensousPeaks)
for index,row in peakListNeg.iterrows():
    unique = True
    for index2,row2 in consensousPeaks.iterrows():
        if abs(getRT(row) - getRT(row2)) < rtTol and 1e6 * np.abs(row["mz"] + 2.0146 - row2["mz"])/(row["mz"] + 2.0146) < ppmTol:
            unique = False
            toAdd[len(toAdd)] = {"Name":row2["Name"],"mz":row["mz"],"rt_start":row2["rt_start"],"rt_end":row2["rt_end"],"Charge":row["Charge"]}
            break
    if unique:
        compoundInd += 1
        toAdd[len(toAdd)] = {"Name":"Compound " + str(compoundInd),"mz":row["mz"],"rt_start":row["rt_start"],"rt_end":row["rt_end"],"Charge":row["Charge"]}

toAdd = pd.DataFrame.from_dict(toAdd,orient="index")
consensousPeaks = pd.concat((consensousPeaks,toAdd),axis=0,ignore_index=True)

In [23]:
consensousPeaks

,Name,mz,rt_start,rt_end,Charge
0,Compound 1,524.265000,0.000000,15.000000,1.0
1,Compound 2,108.080841,0.828070,1.216255,1.0
2,Compound 3,102.127731,6.769517,7.328251,1.0
3,Compound 4,102.127731,6.980431,7.453215,1.0
4,Compound 5,162.112534,5.335723,5.680650,1.0
...,...,...,...,...,...
982,Compound 923,269.121613,1.012359,1.097021,-1.0
983,Compound 924,324.072235,5.060811,5.288692,-1.0
984,Compound 925,190.035751,7.331674,7.617239,-1.0
985,Compound 926,238.891876,10.705984,11.104264,-1.0


In [24]:
consensousPeaks.to_csv("../data/IDX/AX_stab_pool/combined_peak_list.csv")